<a href="https://colab.research.google.com/github/dfukagaw28/ColabNotebooks/blob/main/%E9%9D%92%E7%A9%BA%E6%96%87%E5%BA%AB%E3%81%AE%E3%83%86%E3%82%AD%E3%82%B9%E3%83%88%E3%83%87%E3%83%BC%E3%82%BF%E3%82%92%E7%A2%BA%E8%AA%8D%E3%81%99%E3%82%8B_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 青空文庫のテキストデータを確認する (1)

*   データについて
    *   https://github.com/aozorabunko/aozorabunko 公式テキストデータ（個々のファイル形式は zip）
    *   https://github.com/aozorahack/aozorabunko_text （準公式？）テキストデータ（個々のテキスト形式は txt，文字コードは Shift-JIS）
    *   その他，非公式テキストデータも存在するが，割愛。
*   このノートブックで扱うのは「公式テキストデータ」
*   まとめ
    * 全体的に zip ファイルに 1 個ずつテキストファイルが入っている（例外もある）
    *

In [1]:
!date

Thu Oct 17 11:31:17 AM UTC 2024


*   [青空文庫 新着情報](https://www.aozora.gr.jp/index_pages/whatsnew1.html)

## テキストデータをダウンロードする

*   手元にコピーする。
    *   データサイズが大きい（zip で約 2.5 GB）ため注意。
    *   テキストデータだけが欲しい場合，aozorahack の一括 zip ファイル（約200MB）を利用する方が望ましい。
    *   git には sparse checkout という機能があり，特定のフォルダ以下だけをダウンロードすることもできるが，処理に時間がかかる（3分くらい）

In [2]:
%%time
!curl -sRLO https://github.com/aozorabunko/aozorabunko/archive/refs/heads/master.zip
!ls -l master.zip
!unzip -q master.zip 'aozorabunko-master/cards/*'
!mv aozorabunko-master/cards ./
!rmdir aozorabunko-master

-rw-r--r-- 1 root root 2518036057 Oct 17 11:33 master.zip
CPU times: user 1.14 s, sys: 201 ms, total: 1.34 s
Wall time: 2min 36s


*   全部で約 3 GB

In [3]:
!du -sh cards

3.1G	cards


*   zip ファイルの個数は 17684 個（2024-10-17 現在）

In [4]:
!find cards -type f -name '*.zip' | wc -l

17684


## テキストファイルを含んだ zip ファイルのリストを作成する

In [5]:
!find cards -type f -name '*.zip' | sort > filelist.txt

In [6]:
import pandas as pd

df = pd.read_csv('filelist.txt', header=None, names=['path'])

df

,path
0,cards/000005/files/53194_ruby_44732.zip
1,cards/000005/files/55215_ruby_49076.zip
2,cards/000005/files/55216_ruby_49074.zip
3,cards/000005/files/55217_ruby_49075.zip
4,cards/000005/files/5_ruby_21311.zip
...,...
17679,cards/002265/files/62681_ruby_77884.zip
17680,cards/002265/files/62682_ruby_77911.zip
17681,cards/002265/files/62683_ruby_77947.zip
17682,cards/002265/files/62684_ruby_77978.zip


*   各 path は， `cards/(作家No)/files/(作品No)_(形式)_(連番).zip` のようになっている
    *   例外もある

In [7]:
df['path'].str.split('/').apply(len).value_counts()

,count
path,
4,17684


In [8]:
df['author_id'] = df['path'].str.split('/').str[1]
df['filename'] = df['path'].str.split('/').str[3]

df

,path,author_id,filename
0,cards/000005/files/53194_ruby_44732.zip,000005,53194_ruby_44732.zip
1,cards/000005/files/55215_ruby_49076.zip,000005,55215_ruby_49076.zip
2,cards/000005/files/55216_ruby_49074.zip,000005,55216_ruby_49074.zip
3,cards/000005/files/55217_ruby_49075.zip,000005,55217_ruby_49075.zip
4,cards/000005/files/5_ruby_21311.zip,000005,5_ruby_21311.zip
...,...,...,...
17679,cards/002265/files/62681_ruby_77884.zip,002265,62681_ruby_77884.zip
17680,cards/002265/files/62682_ruby_77911.zip,002265,62682_ruby_77911.zip
17681,cards/002265/files/62683_ruby_77947.zip,002265,62683_ruby_77947.zip
17682,cards/002265/files/62684_ruby_77978.zip,002265,62684_ruby_77978.zip


*   ファイル名については
    *   拡張子はすべて `.zip`
    *   形式は主に ruby か txt だが，例外もある
    *   連番がない場合もある

In [9]:
df['filename'].str[-4:].value_counts()

,count
filename,
.zip,17684


In [10]:
df['stem'] = df['filename'].str[:-4]

In [11]:
df['stem'].str.split('_').apply(len).value_counts()

,count
stem,
3,17316
2,368


In [12]:
df['work_id'] = df['stem'].str.split('_').str[0]
df['format'] = df['stem'].str.split('_').str[1]
df['seq'] = (df['stem'] + '_').str.split('_').str[2]

df

,path,author_id,filename,stem,work_id,format,seq
0,cards/000005/files/53194_ruby_44732.zip,000005,53194_ruby_44732.zip,53194_ruby_44732,53194,ruby,44732
1,cards/000005/files/55215_ruby_49076.zip,000005,55215_ruby_49076.zip,55215_ruby_49076,55215,ruby,49076
2,cards/000005/files/55216_ruby_49074.zip,000005,55216_ruby_49074.zip,55216_ruby_49074,55216,ruby,49074
3,cards/000005/files/55217_ruby_49075.zip,000005,55217_ruby_49075.zip,55217_ruby_49075,55217,ruby,49075
4,cards/000005/files/5_ruby_21311.zip,000005,5_ruby_21311.zip,5_ruby_21311,5,ruby,21311
...,...,...,...,...,...,...,...
17679,cards/002265/files/62681_ruby_77884.zip,002265,62681_ruby_77884.zip,62681_ruby_77884,62681,ruby,77884
17680,cards/002265/files/62682_ruby_77911.zip,002265,62682_ruby_77911.zip,62682_ruby_77911,62682,ruby,77911
17681,cards/002265/files/62683_ruby_77947.zip,002265,62683_ruby_77947.zip,62683_ruby_77947,62683,ruby,77947
17682,cards/002265/files/62684_ruby_77978.zip,002265,62684_ruby_77978.zip,62684_ruby_77978,62684,ruby,77978


## ここまでおさらい

In [13]:
import pandas as pd

df = pd.read_csv('filelist.txt', header=None, names=['path'])

df['author_id'] = df['path'].str.split('/').str[1]
df['filename'] = df['path'].str.split('/').str[3]
df['stem'] = df['filename'].str[:-4]
df['work_id'] = df['stem'].str.split('_').str[0]
df['format'] = df['stem'].str.split('_').str[1]
df['seq'] = (df['stem'] + '_').str.split('_').str[2]

df

,path,author_id,filename,stem,work_id,format,seq
0,cards/000005/files/53194_ruby_44732.zip,000005,53194_ruby_44732.zip,53194_ruby_44732,53194,ruby,44732
1,cards/000005/files/55215_ruby_49076.zip,000005,55215_ruby_49076.zip,55215_ruby_49076,55215,ruby,49076
2,cards/000005/files/55216_ruby_49074.zip,000005,55216_ruby_49074.zip,55216_ruby_49074,55216,ruby,49074
3,cards/000005/files/55217_ruby_49075.zip,000005,55217_ruby_49075.zip,55217_ruby_49075,55217,ruby,49075
4,cards/000005/files/5_ruby_21311.zip,000005,5_ruby_21311.zip,5_ruby_21311,5,ruby,21311
...,...,...,...,...,...,...,...
17679,cards/002265/files/62681_ruby_77884.zip,002265,62681_ruby_77884.zip,62681_ruby_77884,62681,ruby,77884
17680,cards/002265/files/62682_ruby_77911.zip,002265,62682_ruby_77911.zip,62682_ruby_77911,62682,ruby,77911
17681,cards/002265/files/62683_ruby_77947.zip,002265,62683_ruby_77947.zip,62683_ruby_77947,62683,ruby,77947
17682,cards/002265/files/62684_ruby_77978.zip,002265,62684_ruby_77978.zip,62684_ruby_77978,62684,ruby,77978


## 除外すべきファイルたち

In [14]:
skip_files = [
    # 原作者と翻訳者の両方にファイルが存在する（原作者の方を残す）
    'cards/001030/files/47959_ruby_40639.zip',
    'cards/001030/files/47957_ruby_40644.zip',
    'cards/001030/files/47971_txt_40650.zip',
    'cards/001030/files/47896_ruby_49619.zip',

    # 同じ作家の作品が，作家名と本名の２か所に存在する（作家名の方を残す）
    'cards/000975/files/50558_ruby_61314.zip',

    # zip ファイルが壊れて読み込めない
    'cards/001154/files/chihobunkano_shinkensetsu.zip',  # ファイルサイズが 0
    'cards/001562/files/56151_ruby_60063.zip',           # 内容がテキストファイル（zip形式でない）
    'cards/001505/files/58100_txt_60357.zip',            # 壊れて開けない

    # 特殊なテキスト
    'cards/000025/files/kantou_syogo.zip',  # 有島 武郎『或る女』初版本に収められていた、巻頭のホイットマンの詩、及び「書後」のテキスト
    'cards/000065/files/393_etc.zip',  # 九鬼周造『「いき」の構造』 ruby, txt, etc の 3 種類があり，etc は ruby と内容がほぼ同じ

    # zipの内容がテキストファイルでない
    'cards/000050/files/57476_etc_58134.zip',  # PDFファイル
    'cards/000148/files/769_ttz.zip',  # TTZファイル（電子書籍フォーマット）
    'cards/000879/files/128_ttz.zip',  # TTZファイル（電子書籍フォーマット）
    'cards/001021/files/49355_etc_58116.zip',  # 画像のみ
    'cards/001248/files/46751_etc_57109.zip',  # 画像のみ
]

In [15]:
# 指定されたファイルを除外する
df = df.loc[~df['path'].isin(skip_files), :]

df

,path,author_id,filename,stem,work_id,format,seq
0,cards/000005/files/53194_ruby_44732.zip,000005,53194_ruby_44732.zip,53194_ruby_44732,53194,ruby,44732
1,cards/000005/files/55215_ruby_49076.zip,000005,55215_ruby_49076.zip,55215_ruby_49076,55215,ruby,49076
2,cards/000005/files/55216_ruby_49074.zip,000005,55216_ruby_49074.zip,55216_ruby_49074,55216,ruby,49074
3,cards/000005/files/55217_ruby_49075.zip,000005,55217_ruby_49075.zip,55217_ruby_49075,55217,ruby,49075
4,cards/000005/files/5_ruby_21311.zip,000005,5_ruby_21311.zip,5_ruby_21311,5,ruby,21311
...,...,...,...,...,...,...,...
17679,cards/002265/files/62681_ruby_77884.zip,002265,62681_ruby_77884.zip,62681_ruby_77884,62681,ruby,77884
17680,cards/002265/files/62682_ruby_77911.zip,002265,62682_ruby_77911.zip,62682_ruby_77911,62682,ruby,77911
17681,cards/002265/files/62683_ruby_77947.zip,002265,62683_ruby_77947.zip,62683_ruby_77947,62683,ruby,77947
17682,cards/002265/files/62684_ruby_77978.zip,002265,62684_ruby_77978.zip,62684_ruby_77978,62684,ruby,77978


## 作品Noと連番を文字列から整数に変換する

In [16]:
from os import replace
df.loc[df['seq'] == '', 'seq'] = '0'
df.loc[:, 'seq'] = df['seq'].astype(int)
df.loc[:, 'work_id'] = df['work_id'].astype(int)

df

,path,author_id,filename,stem,work_id,format,seq
0,cards/000005/files/53194_ruby_44732.zip,000005,53194_ruby_44732.zip,53194_ruby_44732,53194,ruby,44732
1,cards/000005/files/55215_ruby_49076.zip,000005,55215_ruby_49076.zip,55215_ruby_49076,55215,ruby,49076
2,cards/000005/files/55216_ruby_49074.zip,000005,55216_ruby_49074.zip,55216_ruby_49074,55216,ruby,49074
3,cards/000005/files/55217_ruby_49075.zip,000005,55217_ruby_49075.zip,55217_ruby_49075,55217,ruby,49075
4,cards/000005/files/5_ruby_21311.zip,000005,5_ruby_21311.zip,5_ruby_21311,5,ruby,21311
...,...,...,...,...,...,...,...
17679,cards/002265/files/62681_ruby_77884.zip,002265,62681_ruby_77884.zip,62681_ruby_77884,62681,ruby,77884
17680,cards/002265/files/62682_ruby_77911.zip,002265,62682_ruby_77911.zip,62682_ruby_77911,62682,ruby,77911
17681,cards/002265/files/62683_ruby_77947.zip,002265,62683_ruby_77947.zip,62683_ruby_77947,62683,ruby,77947
17682,cards/002265/files/62684_ruby_77978.zip,002265,62684_ruby_77978.zip,62684_ruby_77978,62684,ruby,77978


## 同じ作品を集約する

*   seq が大きいものを優先する
*   seq が同じであれば，ruby より txt を優先する

In [17]:
df = df.copy()
df['priority'] = ('000000' + df['seq'].astype(str)).str[-6:] + df['format']
df.sort_values('priority')
df = df.groupby('work_id').agg('last').reset_index()

df

,work_id,path,author_id,filename,stem,format,seq,priority
0,2,cards/000012/files/2_ruby_20958.zip,000012,2_ruby_20958.zip,2_ruby_20958,ruby,20958,020958ruby
1,5,cards/000005/files/5_ruby_21311.zip,000005,5_ruby_21311.zip,5_ruby_21311,ruby,21311,021311ruby
2,7,cards/000008/files/7_ruby_20893.zip,000008,7_ruby_20893.zip,7_ruby_20893,ruby,20893,020893ruby
3,8,cards/000009/files/8_ruby_31219.zip,000009,8_ruby_31219.zip,8_ruby_31219,ruby,31219,031219ruby
4,9,cards/000011/files/9_ruby_20330.zip,000011,9_ruby_20330.zip,9_ruby_20330,ruby,20330,020330ruby
...,...,...,...,...,...,...,...,...
17391,62682,cards/002265/files/62682_ruby_77911.zip,002265,62682_ruby_77911.zip,62682_ruby_77911,ruby,77911,077911ruby
17392,62683,cards/002265/files/62683_ruby_77947.zip,002265,62683_ruby_77947.zip,62683_ruby_77947,ruby,77947,077947ruby
17393,62684,cards/002265/files/62684_ruby_77978.zip,002265,62684_ruby_77978.zip,62684_ruby_77978,ruby,77978,077978ruby
17394,62685,cards/002265/files/62685_ruby_78005.zip,002265,62685_ruby_78005.zip,62685_ruby_78005,ruby,78005,078005ruby


## zipファイルにテキストファイルが1つだけ含まれることを確認する

In [18]:
import zipfile

for path in df['path']:
  with zipfile.ZipFile(path, 'r') as zip_ref:
    info_list = zip_ref.infolist()
    info_list = [info for info in info_list if not info.is_dir()]
    info_list = [info for info in info_list if not info.filename.endswith('.png')]
    info_list = [info for info in info_list if not info.filename.endswith('.ini')]
    info_list = [info for info in info_list if not info.filename.startswith('__MACOSX/')]

    if len(info_list) == 1 and info_list[0].filename[-4:].lower() == '.txt':
      pass
    else:
      print(path)
      print(info_list)
      print()

## テキストファイルの名前を取得する

In [19]:
def get_text_filename(path):
  with zipfile.ZipFile(path, 'r') as zip_ref:
    info_list = zip_ref.infolist()
    info_list = [info for info in info_list if not info.is_dir()]
    info_list = [info for info in info_list if not info.filename.endswith('.png')]
    info_list = [info for info in info_list if not info.filename.endswith('.ini')]
    info_list = [info for info in info_list if not info.filename.startswith('__MACOSX/')]
    assert len(info_list) == 1 and info_list[0].filename[-4:].lower() == '.txt'
    return info_list[0].filename

df['text_filename'] = df['path'].apply(get_text_filename)

## 集計結果をファイルに保存する

In [20]:
df = df[['path', 'author_id', 'work_id', 'seq', 'text_filename']]
df.to_csv('filelist.csv', index=False)

In [21]:
!head filelist.csv

path,author_id,work_id,seq,text_filename
cards/000012/files/2_ruby_20958.zip,000012,2,20958,sanjusanno_shi.txt
cards/000005/files/5_ruby_21311.zip,000005,5,21311,aibiki.txt
cards/000008/files/7_ruby_20893.zip,000008,7,20893,akagaeru.txt
cards/000009/files/8_ruby_31219.zip,000009,8,31219,akage_renmei.txt
cards/000011/files/9_ruby_20330.zip,000011,9,20330,akatsukito_yubeno_uta.txt
cards/000012/files/10_ruby_20387.zip,000012,10,20387,akiwa_sabishii.txt
cards/000013/files/11_ruby_20061.zip,000013,11,20061,akino_hitomi.txt
cards/000014/files/12_ruby_22756.zip,000014,12,22756,akumano_shita.txt
cards/000879/files/13_ruby_1960.zip,000879,13,1960,jipponno_hari.txt
